# Creating and Visualizing DEMs from LIDAR points

_**Caitlin Haedrich and Pratikshya Regmi**, North Carolina State University_

In this notebook we will:
* Create high-quality DEMs from LiDAR point clouds and compute topographic parameters
* Create webmap visualizations


## 1. Startup

Import Python standard library and IPython packages we need. Start GRASS session in Nags Head project.

In [ ]:
import subprocess
import sys
from pathlib import Path

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True, shell=False).strip()
)

import grass.script as gs
import grass.jupyter as gj

gj.init("./nags_head/PERMANENT");

In [ ]:
!g.region -p

## 3. Creating DEMs

In [ ]:
lidar_files = sorted(Path('./data/').glob('*.las'))
lidar_files

### Mask low density areas

Interpolating a DEM from LiDAR points is only meaningful where there are adequate point coverage. So, our first step will be to mask areas that have low point densities before we interpolate. This also will make our interpolation run faster!

In [ ]:
!g.region raster="naip_2020" res=5

In [ ]:
gs.run_command("r.in.pdal", input="./data/JR_2014.las", output="JR_2014_raw", method="n", flags="w")
gs.mapcalc(exp="JR_2014_mask=if(JR_2014_raw == 0, null(), 1 )")

In [ ]:
mask = gj.Map()
mask.d_rast(map="naip_2014")
mask.d_rast(map="JR_2014_mask")
mask.show()

In [ ]:
!g.region res=0.5
!r.mask raster=JR_2014_mask

### Creating DEMs with Binning

In [ ]:
# Binning
!g.region res=2

In [ ]:
gs.run_command("r.in.pdal", input="./data/JR_2014.las", output="JR_2014", method="mean", flags="w")

In [ ]:
!r.mask -r

For you can also compute DEMs with a higher resolution than the point sampling distance using splines with either the [v.surf.rst](https://grass.osgeo.org/grass83/manuals/v.surf.rst.html) or [v.surf.bspline](https://grass.osgeo.org/grass83/manuals/v.surf.bspline.html) tools.

See Section 2.3.3 of [Hardin et al (2014)](https://link.springer.com/chapter/10.1007/978-1-4939-1835-5_2#Sec3) for more.

In [ ]:
elev = gj.Map()
elev.d_rast(map="naip_2014")
elev.d_rast(map="JR_2014")
elev.show()

In [ ]:
fig = gj.InteractiveMap(width=800)
fig.add_raster("naip_2014")
fig.add_raster("JR_2014")
fig.add_layer_control()
fig.show()